In [1]:
from pynq import Overlay
import numpy as np
from pynq import MMIO
import os 
from pynq import allocate
import math
import pynq.lib.dma
import time
from pynq import PL
import asyncio
from pynq import Interrupt

# 导入工程文件

In [3]:
overlay = Overlay("./Windows.bit")

In [8]:
overlay?

In [6]:
#-------------------------------------------滑动窗口-----------------------------------------
Windows_Data_address=0x4000_1000
Windows_Data_range=0xFFF
Windows_mmio=MMIO(Windows_Data_address,Windows_Data_range)
Data_horizontal_address=0x0;
Data_vertical_address=0x4;
Kernel_size_address=0x8;
Stride_address=0xc;
Refresh_address=0x10;

Data_vertical  =28
Data_horizontal=28
Stride          =2
Refresh         =0

In [12]:
Windows_Convert=overlay.Windows_Data_Convert_0 
dma_result=overlay.axi_dma_0                   
dma_picture=overlay.axi_dma_1             
DATA_SEND = dma_picture.sendchannel
RESULT_RECV= dma_result.recvchannel


In [11]:
def Sliding_window_settings(Data_horizontal,Data_vertical,Kernel_size,Stride=2,Refresh=0):
    Windows_mmio.write(Kernel_size_address,Kernel_size)
    Windows_mmio.write(Data_vertical_address,Data_vertical)
    Windows_mmio.write(Data_horizontal_address,Data_horizontal)
    Windows_mmio.write(Stride_address,Stride)
    Windows_mmio.write(Refresh_address,Refresh)
    
    Kernel_size=Windows_mmio.read(Kernel_size_address)
    Data_vertical=Windows_mmio.read(Data_vertical_address)
    Data_horizontal=Windows_mmio.read(Data_horizontal_address)
    Stride=Windows_mmio.read(Stride_address)
    Refresh=Windows_mmio.read(Refresh_address)
    print("滑窗模块 Kernel_size核心大小      =",Kernel_size)
    print("滑窗模块 Data_vertical图像纵向长  =",Data_vertical)
    print("滑窗模块 Data_horizontal图像横向长=",Data_horizontal)
    print("滑窗模块 Stride步进               =",Stride)
    print("滑窗模块 Refresh刷新              =",Refresh)
  

In [34]:
# input_picture_buffer 初始化
Data_vertical=12
Data_horizontal=Data_vertical
Kernel_Size=4
Stride_Set=1
Windows_refresh=0

input_picture_data_size = Data_horizontal*Data_vertical
input_picture_buffer = allocate(shape=(input_picture_data_size,), dtype=np.int8)
for i in range(input_picture_data_size):
    input_picture_buffer[i] = i%32+1
data = np.ones((input_picture_data_size,), dtype=np.int8)

Sliding_window_settings(Data_horizontal,Data_vertical,Kernel_Size,Stride_Set,Windows_refresh)

print("----------------------INPUT---ACT---------------------") 
for j in range(Data_horizontal):
    print("\n")
    for k in range(Data_vertical):
            print((input_picture_buffer[j*Data_horizontal+k]),"  ", end="")
print("\n")           

DATA_SEND.transfer(input_picture_buffer)
DATA_SEND.wait()

滑窗模块 Kernel_size核心大小      = 4
滑窗模块 Data_vertical图像纵向长  = 12
滑窗模块 Data_horizontal图像横向长= 12
滑窗模块 Stride步进               = 1
滑窗模块 Refresh刷新              = 0
----------------------INPUT---ACT---------------------


1   2   3   4   5   6   7   8   9   10   11   12   

13   14   15   16   17   18   19   20   21   22   23   24   

25   26   27   28   29   30   31   32   1   2   3   4   

5   6   7   8   9   10   11   12   13   14   15   16   

17   18   19   20   21   22   23   24   25   26   27   28   

29   30   31   32   1   2   3   4   5   6   7   8   

9   10   11   12   13   14   15   16   17   18   19   20   

21   22   23   24   25   26   27   28   29   30   31   32   

1   2   3   4   5   6   7   8   9   10   11   12   

13   14   15   16   17   18   19   20   21   22   23   24   

25   26   27   28   29   30   31   32   1   2   3   4   

5   6   7   8   9   10   11   12   13   14   15   16   



In [35]:
# def Show_result_data(select_mode):
#     global matrix_conv
#     global result_buffer 
#-----------------------------------------卷积/卷积+激活-------------------------------------    
CONV_hang_size=math.floor((Data_horizontal-Kernel_Size)/Stride_Set+1)
print("结果输出一行个数---------",CONV_hang_size)
data_size=CONV_hang_size*CONV_hang_size
print("结果输出总个数---------",result_data_size)
result_data_size=data_size*16#多的空闲位置  16bit*8  
result_data_size=math.ceil(result_data_size) 
result_buffer = allocate(shape=(result_data_size,), dtype=np.int8)
#-------------------------------获取计算结果-------------------------
RESULT_RECV.transfer(result_buffer)
RESULT_RECV.wait() 
# matrix_conv = np.zeros((16, result_data_size), dtype=np.int8)
# for j in range(int(result_data_size/16)):
#     for h in range(16):
#         matrix_conv[h][j]=result_buffer[j*16+h]

for h in range(data_size):
    print("----第", h+1,"个滑动窗结果----","  ", end="")#
    for i in range(int(Kernel_Size)):
        print("") 
        for j in range(int(Kernel_Size)):
            print((result_buffer[h*16+i*Kernel_Size+j]),"  ", end="")
    print("")   


结果输出一行个数--------- 9
结果输出总个数--------- 1296
----第 1 个滑动窗结果----   
1   2   3   4   
13   14   15   16   
25   26   27   28   
5   6   7   8   
----第 2 个滑动窗结果----   
2   3   4   5   
14   15   16   17   
26   27   28   29   
6   7   8   9   
----第 3 个滑动窗结果----   
3   4   5   6   
15   16   17   18   
27   28   29   30   
7   8   9   10   
----第 4 个滑动窗结果----   
4   5   6   7   
16   17   18   19   
28   29   30   31   
8   9   10   11   
----第 5 个滑动窗结果----   
5   6   7   8   
17   18   19   20   
29   30   31   32   
9   10   11   12   
----第 6 个滑动窗结果----   
6   7   8   9   
18   19   20   21   
30   31   32   1   
10   11   12   13   
----第 7 个滑动窗结果----   
7   8   9   10   
19   20   21   22   
31   32   1   2   
11   12   13   14   
----第 8 个滑动窗结果----   
8   9   10   11   
20   21   22   23   
32   1   2   3   
12   13   14   15   
----第 9 个滑动窗结果----   
9   10   11   12   
21   22   23   24   
1   2   3   4   
13   14   15   16   
----第 10 个滑动窗结果----   
13   14   15   16   
25   26   27  